# Scapy 실습

In [1]:
from scapy.all import *

In [2]:
# Scapy에서 제공하는 기능들
lsc()

IPID_count            : Identify IP id values classes in a list of packets
arp_mitm              : ARP MitM: poison 2 target's ARP cache
arpcachepoison        : Poison targets' ARP cache
arping                : Send ARP who-has requests to determine which hosts are up
arpleak               : Exploit ARP leak flaws, like NetBSD-SA2017-002.
bind_layers           : Bind 2 layers on some specific fields' values.
bridge_and_sniff      : Forward traffic between interfaces if1 and if2, sniff and return
chexdump              : Build a per byte hexadecimal representation
computeNIGroupAddr    : Compute the NI group Address. Can take a FQDN as input parameter
corrupt_bits          : Flip a given percentage (at least one bit) or number of bits
corrupt_bytes         : Corrupt a given percentage (at least one byte) or number of bytes
defrag                : defrag(plist) -> ([not fragmented], [defragmented],
defragment            : defragment(plist) -> plist defragmented as much as possible 
dhcp_r

## 패킷 구조

> `ls(obj, case_sensitive, verbose)`

    - obj: 패킷 혹은 패킷 이름 
    - case_sensitive (bool): obj가 스트링일 경우 대소문자 구분 여부
    - verbose (bool): 추가적인 정보 로깅 여부

### IPv4 패킷 구조

<img src="https://www.dropbox.com/s/7dn0tzw9nzt2dcw/ip.png?raw=1" width="466" height="216"/>

In [3]:
ls(IP(), verbose=True)

version    : BitField  (4 bits)                  = 4               ('4')
ihl        : BitField  (4 bits)                  = None            ('None')
tos        : XByteField                          = 0               ('0')
len        : ShortField                          = None            ('None')
id         : ShortField                          = 1               ('1')
flags      : FlagsField                          = <Flag 0 ()>     ('<Flag 0 ()>')
               MF, DF, evil
frag       : BitField  (13 bits)                 = 0               ('0')
ttl        : ByteField                           = 64              ('64')
proto      : ByteEnumField                       = 0               ('0')
chksum     : XShortField                         = None            ('None')
src        : SourceIPField                       = '127.0.0.1'     ('None')
dst        : DestIPField                         = '127.0.0.1'     ('None')
options    : PacketListField                     = []              ('[

### TCP 패킷 구조

<img src="https://www.dropbox.com/s/wuo7g9vjzydto8s/tcp.png?raw=1" width="521" height="225"/>

In [4]:
ls(TCP(), verbose=True)

sport      : ShortEnumField                      = 20              ('20')
dport      : ShortEnumField                      = 80              ('80')
seq        : IntField                            = 0               ('0')
ack        : IntField                            = 0               ('0')
dataofs    : BitField  (4 bits)                  = None            ('None')
reserved   : BitField  (3 bits)                  = 0               ('0')
flags      : FlagsField                          = <Flag 2 (S)>    ('<Flag 2 (S)>')
               F, S, R, P, A, U, E, C, N
window     : ShortField                          = 8192            ('8192')
chksum     : XShortField                         = None            ('None')
urgptr     : ShortField                          = 0               ('0')
options    : TCPOptionsField                     = []              ("b''")


- `/` 연산자로 패킷을 쌓을 수 있음
    - Data Link Layer / Network Layer / Transport Layer


In [5]:
packet = Ether() / IP() / TCP()
print(packet.summary())
print(packet[IP].summary())

Ether / IP / TCP 127.0.0.1:ftp_data > 127.0.0.1:http S
IP / TCP 127.0.0.1:ftp_data > 127.0.0.1:http S


- 각 필드에 원하는 값 저장 가능
- 각 필드에 접근도 가능

In [6]:
p = Ether() / IP(dst="www.google.com") / TCP(flags='F')
p.summary()

'Ether / IP / TCP 141.223.140.37:ftp_data > Net("www.google.com/32"):http F'

### 패킷 캡처

> `sniff(iface, prn, filter, count, timeout) -> PacketList`

    - iface: 스니핑할 네트워크 인터페이스 (ifconfig로 확인)
    - prn: 각각의 캡처된 패킷의 콜백 함수. 콜백 함수는 캡처한 패킷을 인자로 받아야 함.
    - filter: BPF (Berkeley Packet Filter) 표현식
    - count: 캡처할 패킷 수
    - timeout: 캡처할 기간 (초) 

In [7]:
s = sniff(iface="enp0s31f6", filter="tcp", count=10)
s

<Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>

In [8]:
for packet in s:
    print(packet.summary())

Ether / IP / TCP 47.254.127.84:47043 > 141.223.140.27:eklogin S / Padding
Ether / IP / TCP 141.223.140.37:49370 > 54.150.70.46:https PA / Raw
Ether / IP / TCP 54.150.70.46:https > 141.223.140.37:49370 PA / Raw
Ether / IP / TCP 141.223.140.37:49370 > 54.150.70.46:https A
Ether / IP / TCP 178.32.43.185:60000 > 141.223.122.167:10012 S / Padding
Ether / IP / TCP 113.130.247.67:50582 > 141.223.140.27:2323 S / Padding
Ether / IP / TCP 183.136.225.10:15035 > 141.223.122.153:184 S / Padding
Ether / IP / TCP 89.248.164.165:44054 > 141.223.122.167:5031 S / Padding
Ether / IP / TCP 141.223.140.37:44028 > 152.195.38.76:http A
Ether / IP / TCP 141.223.140.37:44022 > 152.195.38.76:http A


In [9]:
s[0]

<Ether  dst=50:9a:4c:c0:86:a5 src=6c:4e:f6:85:90:46 type=IPv4 |<IP  version=4 ihl=5 tos=0x14 len=44 id=0 flags=DF frag=0 ttl=51 proto=tcp chksum=0x7e6b src=47.254.127.84 dst=141.223.140.27 |<TCP  sport=47043 dport=eklogin seq=842717603 ack=0 dataofs=6 reserved=0 flags=S window=1024 chksum=0xfefe urgptr=0 options=[('MSS', 1460)] |<Padding  load='\x00\x00' |>>>>

In [10]:
s[0].show()

###[ Ethernet ]### 
  dst       = 50:9a:4c:c0:86:a5
  src       = 6c:4e:f6:85:90:46
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x14
     len       = 44
     id        = 0
     flags     = DF
     frag      = 0
     ttl       = 51
     proto     = tcp
     chksum    = 0x7e6b
     src       = 47.254.127.84
     dst       = 141.223.140.27
     \options   \
###[ TCP ]### 
        sport     = 47043
        dport     = eklogin
        seq       = 842717603
        ack       = 0
        dataofs   = 6
        reserved  = 0
        flags     = S
        window    = 1024
        chksum    = 0xfefe
        urgptr    = 0
        options   = [('MSS', 1460)]
###[ Padding ]### 
           load      = '\x00\x00'



In [11]:
hexdump(s[0])

0000  50 9A 4C C0 86 A5 6C 4E F6 85 90 46 08 00 45 14  P.L...lN...F..E.
0010  00 2C 00 00 40 00 33 06 7E 6B 2F FE 7F 54 8D DF  .,..@.3.~k/..T..
0020  8C 1B B7 C3 08 39 32 3A D9 A3 00 00 00 00 60 02  .....92:......`.
0030  04 00 FE FE 00 00 02 04 05 B4 00 00              ............


### 인터페이스 리스트 출력

In [13]:
import socket

iface_list = socket.if_nameindex()
iface_list

[(1, 'lo'), (2, 'enp2s0'), (3, 'enp0s31f6')]

### 패킷 파일 저장하기

> `wrpcap(filename, pkt)`

    - filename: 저장할 pcap, pcapng 파일 이름
    - pkt: Packet 혹은 PacketList
    - append (bool): pcap 파일에 이어서 작성하고 싶으면 True 

In [14]:
wrpcap(filename="10.pcap", pkt=s)

<img src="https://www.dropbox.com/s/bhzrdq7k6w71yvd/ex2-1.png?raw=1" width="493" height="257"/>

In [15]:
# 콜백함수 예시
def process_packet(packet):
    wrpcap(pcap_file, packet, append=True) # 

### 패킷 파일 불러오기

> `rdpcap(filename) -> PacketList`

    - filename: 불러올 pcap, pcapng 파일 이름 

In [16]:
pcap = rdpcap("ex1_jokyo.pcap")

FileNotFoundError: [Errno 2] No such file or directory: 'ex1_jokyo.pcap'

### 패킷 정보 확인

In [ ]:
print(f"Total number of packets: {len(pcap)}")
print(f"Bytes of packet: {len(pcap[0])}")
print(f"Packet arrival time in Unix time: {pcap[0].time}")
print(f"Packet uses UDP: {pcap[0].haslayer(UDP)}")
print(f"Src/Dst port of the packet: {pcap[0][UDP].sport}/{pcap[0][UDP].dport}")

<img src="https://www.dropbox.com/s/b25w53uvusz063u/ex2-2.png?raw=1" width="487" height="265"/>

In [ ]:
# load_contrib("gtp")
gtp = rdpcap("ex3_gtp.pcap")

In [ ]:
a = gtp[0]
a.summary()

In [ ]:
ls(GTP_U_Header())

In [ ]:
ls(a[GTP_U_Header][IP])

<img src="https://www.dropbox.com/s/00f8pqjjanl9al7/ex3.png?raw=1" width="510" height="281"/>